In [1]:

import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10


In [2]:
from azureml.core import Workspace
#ws=Workspace.get(name='zAML',subscription_id='my_subscription_id')


In [ ]:
# Create the configuration file.
#ws.write_config()

# Use this code to load the workspace from 
# other scripts and notebooks in this directory.
# ws = Workspace.from_config()

In [3]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      #'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/nbuser/library/aml_config/config.json
Workspace name: zAML
Azure region: westeurope
Resource group: AMLrg


In [4]:

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [5]:

from azureml.core import Datastore

ds = ws.get_default_datastore()
print("Using the default datastore for training data: ")
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

Using the default datastore for training data: 
workspaceblobstore AzureBlob zaml5744793264 azureml-blobstore-9d836814-f895-4a65-af78-59f755a38c85


In [6]:
 
from azureml.core import Datastore
# only need to do it once
ds2 = Datastore.register_azure_file_share(workspace=ws, 
                                         datastore_name='give_a_name_of_your_choice', 
                                         file_share_name='file_share_name',
                                         account_name='your_storage_acc_name', 
                                         account_key='your_storage_acc_key',
                                         create_if_not_exists=False)
                                         

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-28T13:46:20.126000+00:00', 'creationTime': '2019-01-08T10:23:55.033355+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-08T10:25:35.793472+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [8]:
import os

project_folder = './keras-ctscan-exp-feb28'
os.makedirs(project_folder, exist_ok=True)

In [9]:
import shutil

#shutil.copy('keras_cnn_dicom.py', project_folder)
shutil.copy('keras_cnn_pydicom.py', project_folder)


'./keras-ctscan-exp-feb28/keras_cnn_pydicom.py'

In [11]:
cat ./keras-ctscan-exp-feb28/keras_cnn_pydicom.py

from __future__ import print_function
import argparse
import pydicom
from matplotlib import pyplot, cm
import os
import sys
import numpy as np
import pandas as pd
import scipy
import keras
from keras.models import Sequential
from keras.layers import AveragePooling2D , Convolution2D , Flatten ,Dense, MaxPooling2D, Conv2D
from keras.preprocessing import utils
from keras.preprocessing.image import ImageDataGenerator


def get_data(dicom_dir):
    #resize the image to desired resolution
    #print("dicom_dir",os.listdir(dicom_dir), dicom_dir)
    xsize = 256; ysize = 256
    
    data = np.zeros((xsize, ysize, 100))
    #print("dicom_dir",os.listdir(dicom_dir), dicom_dir)
    for i, s in enumerate(os.listdir(dicom_dir)):
    
        img = np.array(pydicom.read_file(dicom_dir+ s).pixel_array)
        xscale = xsize/img.shape[0]
        yscale = ysize/img.shape[1]
        data[:,:,i] = scipy.ndimage.interpolation.zoom(img, [xscale, yscale])
    #returning a numpy array of shape 100,256,256


In [12]:
from azureml.core import Experiment

experiment_name = 'keras-tf-exp-feb28'
experiment = Experiment(ws, name=experiment_name)

In [13]:

ds2.path()

$AZUREML_DATAREFERENCE_ctscands

In [14]:
from azureml.train.dnn import TensorFlow 
script_params={
    '--data': ds2.path(),
    '--epoch': 1,
    '--save_model':'/outputs'
}

estimator = TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='keras_cnn_pydicom.py',
                      script_params=script_params,
                      node_count=1,
                      process_count_per_node=1,
                      #distributed_backend='mpi',    
                      pip_packages=['pydicom','keras','scikit-image','scikit-learn','scipy','argparse',
                                    'opencv-contrib-python-headless','pillow','numpy==1.14.5', 'pandas','matplotlib'],
                      #custom_docker_base_image='zecharpy/tfgpupy3:pydicom',
                      use_gpu=True)

In [15]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: keras-tf-exp-feb28,
Id: keras-tf-exp-feb28_1551369917_43572d77,
Type: azureml.scriptrun,
Status: Starting)


In [16]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [17]:
run.wait_for_completion(show_output=True)

RunId: keras-tf-exp-feb28_1551369917_43572d77

Streaming azureml-logs/20_image_build_log.txt

2019/02/28 16:05:40 Using acb_vol_a95ff98d-830e-483d-be27-1fa648e114e8 as the home volume
2019/02/28 16:05:40 Creating Docker network: acb_default_network, driver: 'bridge'
2019/02/28 16:05:41 Successfully set up Docker network: acb_default_network
2019/02/28 16:05:41 Setting up Docker configuration...
2019/02/28 16:05:43 Successfully set up Docker configuration
2019/02/28 16:05:43 Logging in to registry: zaml3818199666.azurecr.io
2019/02/28 16:05:47 Successfully logged into zaml3818199666.azurecr.io
2019/02/28 16:05:47 Executing step ID: acb_step_0. Working directory: '', Network: 'acb_default_network'
2019/02/28 16:05:47 Obtaining source code and scanning for dependencies...
2019/02/28 16:05:48 Successfully obtained source code and scanned for dependencies
2019/02/28 16:05:48 Launching container with name: acb_step_0
Sending build context to Docker daemon  40.96kB

Step 1/13 : FROM mcr.micro

  Stored in directory: /root/.cache/pip/wheels/90/db/f8/2c3101f72ef1ad434e4662853174126ce30201a3e163dcbeca
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built absl-py termcolor gast pyyaml networkx pathspec toolz pycparser
tensorflow-gpu 1.10.0 has requirement numpy<=1.14.5,>=1.13.3,


Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
/mnt/batch/tasks/shared/LS_root/jobs/zaml/azureml/keras-tf-exp-feb28_1551369917_43572d77/mounts/ctscands
check the dicom file shape should be 100, 256,256 (100, 256, 256)
keras_cnn_pydicom.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  classifier.add(Conv2D(64, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:96: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  classifier.add(Conv2D(128, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  classifier.add(Conv2D(256, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:118: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Ke

100/100 [==============================] - 15s 154ms/step - loss: 0.5694 - acc: 0.7325 - val_loss: 0.2527 - val_acc: 0.9000


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.10098147392272949 seconds

Execution Summary
RunId: keras-tf-exp-feb28_1551369917_43572d77



{'runId': 'keras-tf-exp-feb28_1551369917_43572d77',
 'target': 'gpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2019-02-28T16:20:56.883725Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'bc15d3aa-34f1-4511-ad24-ccdc1deb9c90'},
 'runDefinition': {'Script': 'keras_cnn_pydicom.py',
  'Arguments': ['--data',
   '$AZUREML_DATAREFERENCE_ctscands',
   '--epoch',
   '1',
   '--save_model',
   '/outputs'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'gpucluster',
  'DataReferences': {'ctscands': {'DataStoreName': 'ctscands',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',